
## Text Named Entity Similarity Score (evaluation) <br>
                                                 
In the following, we compute the similarity scores for named entities extracted from the translated body text of the articles in the evaluation dataset. <br>

One can argue that extracting named entities after translation is a risky procedure (e.g. translating the German city name _"Essen"_ to the word _"Food"_). We keep track of these issues in **insert DIRECTORY or NAME OF FILE**

In [16]:
import pandas as pd
import spacy 
nlp = spacy.load('en_core_web_sm') 

In [27]:
# 'complete' means no feature (title,keywords, text) is missing from the scrapped json files
# as we feed these features into the neural network, entries missing at least one of them
# cannot be used 
df = pd.read_csv('eval/_EVAL_details_complete_in_df.csv')

In [28]:
df.columns

Index(['pair_id', 'url1_lang', 'url2_lang', 'title1', 'title2', 'keywords1',
       'keywords2', 'text1', 'text2', 'Geography', 'Entities', 'Time',
       'Narrative', 'Overall', 'Style', 'Tone', 'translated_body1',
       'translated_body2'],
      dtype='object')


### Use overlap score for named entities <br>

As a measure of similarity for named entities we use the overlap coefficient. The chosen classes of named entities exclude the ones like _"CARDINAL"_, _"DATE"_, _"MONEY"_, _"ORDINAL"_, _"PERCENT"_, _"QUANTITY"_, and _"TIME"_. <br><br>

The reason for this is that the normalization of NEs in these categories might prove problematic. For instance, understanding that _"20/3/22"_ and _"the previous Sunday"_ correspond to the same date is not obvious. Moreover, we agreed that named entities, such as the ones referring to proper nouns of people or places, might be enough to capture the basic data of the story.

In [30]:
ignore_labels = ['CARDINAL','DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME']

def overlapScore(key1, key2):
    key1 = set(key1)
    key2 = set(key2)
    interset = key1.intersection(key2)
    union = key1.union(key2)
    if len(key1) <1 or len(key2) <1 or len(union) < 1 or len(interset)<1:
        return 0
    return (len(interset)/(len(union)))


text_key_df = pd.DataFrame(columns = ["pair_id","url1_lang", "url2_lang","text1","text2","translated_body1","translated_body2","key1","key2", "key_score"])

for index, row in df.iterrows(): 
    body1 = str(row['translated_body1'])
    body2 = str(row['translated_body2'])
    doc1 = nlp(body1) 
    doc2 = nlp(body2) 
    ner1 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc1.ents if ent.label_ not in ignore_labels]
    ner2 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc2.ents if ent.label_ not in ignore_labels]
    score = overlapScore(ner1, ner2)
    if score > 0:
        print('---------------------',index,'-----------------------')
        print(body1[:20])
        print(ner1[:3])
        print(body2[:20])
        print(ner2[:3])
        print(score)
    
    pair = str(row['pair_id'])
    entry = {"pair_id": pair,"url1_lang":row['url1_lang'],"url2_lang":row['url2_lang'],"text1":row["text1"],"text2":row["text2"]
             ,"translated_body1":row['translated_body1'],"translated_body2":row['translated_body2'],"key1":ner1,"key2":ner2,"key_score":score}
    text_key_df = text_key_df.append(entry, ignore_index = True) ####

--------------------- 0 -----------------------
MARTINSBURG, W.Va. —
['martinsburg', 'wva', 'new year’s day']
PORT-AU-PRINCE, Hait
['haiti', 'haitian', 'jovenel moïse']
0.045454545454545456
--------------------- 1 -----------------------
Uber has sold its on
['india', 'zomato', 'china']
Rapid digitisation a
['india', 'google', 'boston consulting group']
0.12195121951219512
--------------------- 2 -----------------------
BENGALURU (Reuters) 
['bengaluru', 'reuters', 'india']
BANGALORE: India pla
['india', 'indian space research organisation', 'k sivan']
0.3181818181818182
--------------------- 4 -----------------------
From The Guardian

F
['guardian', 'boeing', 'whole foods']
Police have arrested
['orange country', 'ny', 'jews']
0.03125
--------------------- 5 -----------------------
An exhibition outlin
['spike island', 'the war of independence', 'cork harbour']
Taoiseach Leo Varadk
['taoiseach leo varadkar', 'state', 'new year']
0.1282051282051282
--------------------- 7 ------------

--------------------- 63 -----------------------


George Mason Natio
['tim evanson', 'aristocracy', 'monarchy']
Karl R. H. Frick on 
['karl r h frick', 'karl r h frick’s die erleuchteten', 'ende des 18']
0.005847953216374269
--------------------- 64 -----------------------
DETROIT – Police sai
['detroit', 'detroit', 'oakland']
LIVONIA (WWJ) - The 
['wwj', 'detroit', 'the taco bell']
0.35
--------------------- 67 -----------------------
(CNN) Much has been 
['cnn', 'donald trump', 'christianity']
Who Cares: OZY's 202
['cares', 'ozy', 'cares']
0.1282051282051282
--------------------- 69 -----------------------
2019: Oregon Interna
['oregon international air show', 'mcminnville', 'hillsboro']
2019: Deputies wound
['hagg lake', 'gaston', "the news times'"]
0.13513513513513514
--------------------- 71 -----------------------
Text Size: A- A+

Du
['iraqi', 'us embassy', 'baghdad']
Iran’s Supreme Leade
['iran', 'supreme leader', 'ayatollah ali khamenei']
0.25
--------------------- 73 -------

--------------------- 132 -----------------------
As Northern Ireland 
['northern ireland', 'arlene foster', 'northern ireland']
Irish Foreign Affair
['irish', 'simon coveney', 'arlene foster']
0.23529411764705882
--------------------- 133 -----------------------
Sign up to FREE emai
['free', 'cornwalllive   court insider subscribe thank', 'invalid email']
Sign up to FREE dail
['free', 'invalid email', 'new year']
0.012422360248447204
--------------------- 135 -----------------------
QNA

A two-day works
['qna', 'texas a&m university', 'qatar']
(MENAFN - The Penins
['menafn   the peninsula', 'qatar', 'ooredoo']
0.11428571428571428
--------------------- 137 -----------------------
From Common Dreams


['us', 'dmca\n\n\n\nimpeachment', 'trump']


Protesters burn pr
['the us embassy', 'baghdad', 'khalid mohammed/ap']
0.14414414414414414
--------------------- 138 -----------------------
( MENAFN - Saudi Pre
['menafn   saudi press agency', 'riyadh', 'salman bin']
(MENAFN - Saudi Pres
['mena

--------------------- 189 -----------------------
A woman accused of s
['jewish', 'brooklyn', 'tiffany harris']
A Brooklyn miscreant
['brooklyn', 'orthodox', 'jewish']
0.2647058823529412
--------------------- 190 -----------------------
New Delhi: Delhi Dep
['new delhi', 'delhi', 'manish sisodia']
Guwahati: The Assam 
['guwahati', 'esma', 'the essential services maintenance (assam) act']
0.2916666666666667
--------------------- 191 -----------------------
LOS ANGELES (AFP) - 
['los angeles', 'american', 'syd mead']
LOS ANGELES — The ce
['los angeles', 'southern california’s', 'the central coast']
0.04
--------------------- 192 -----------------------
New Delhi (Sputnik):
['new delhi', 'sputnik', 'india']
India seeking to bec
['india', 'russia', 'the united states']
0.3333333333333333
--------------------- 194 -----------------------
With its healthcare 
['the middle east', 'qatar', 'legatum institute']
Narrowing down a lis
['us', 'barack obama', 'obama']
0.03773584905660377
-----------

--------------------- 252 -----------------------


(Image by SANA) De
['dmca', 'syrians', 'sunnis']
The Archbishop of Ca
['the archbishop of canterbury', 'justin portal welby', 'christians']
0.10526315789473684
--------------------- 253 -----------------------
From Truthdig



Car
['truthdig\n\n\n\ncarbon', 'brian boucheron / flickr', 'the us house of representatives']
As we head into the 
['the roaring twenties 20', 'senate', 'donald trump']
0.07142857142857142
--------------------- 254 -----------------------
Hours after North Ko
['north korean', 'kim jong un', 'donald trump']
North Korean leader 
['north korean', 'kim jong un', 'north korean']
0.4
--------------------- 255 -----------------------
‘Tell them that we C
['christians', 'east', 'west']
» 05/03/2010 09:56


['christian', 'mosul', 'the university of mosul']
0.12195121951219512
--------------------- 256 -----------------------
HYDERABAD: A 37-year
['chennai', 'panjagutta', 'osmania general hospital']
The 37-year-old woma
[

...

--------------------- 2384 -----------------------
The coronavirus has 
['austria', 'tyrol', 'switzerland']
Shortly before the s
['austria', 'munich', 'germany']
0.16071428571428573
--------------------- 2385 -----------------------
Berlin / Grünheide (
['berlin / grünheide', 'us', 'berlin']
Tesla Inc. (NASDAQ: 
['tesla inc', 'nasdaq', 'berlin']
0.125
--------------------- 2386 -----------------------
Interview «For inter
['michel rochat', 'the hotelfachschule lausanne', 'china']
COVID-19 is a pandem
['covid 19', 'france', 'chinese']
0.11428571428571428
--------------------- 2388 -----------------------
The Israeli Foreign 
['israeli', 'israel katz', 'the world forum']
cnxps.cmd.push(funct
['cnxps', 'iran', 'benjamin netanyahu']
0.2727272727272727
--------------------- 2389 -----------------------
Nairobi (dpa) - in K
['nairobi', 'kenya', 'kenyans']
press release  For 4
['juliet m', 'kenyan', 'nairobi']
0.14285714285714285
--------------------- 2390 -----------------------
Health Minis

--------------------- 2434 -----------------------
In the midst of the 
['meghan markle', 'prince harry', 'george floyd']
Sign up to FREE dail
['free', 'invalid email', 'meghan markle’s']
0.24242424242424243
--------------------- 2435 -----------------------
Image: Pixnio.com by
['bicanski', 'italian', 'kornelia kirchweger']
A distressing video 
['bergamo', 'italy', 'sky news']
0.23809523809523808
--------------------- 2436 -----------------------
The crisis summit in
['wef', 'us', 'donald trump']
Davos forum: Trump t
['iranian', 'klaus schwab', 'founder']
0.2236842105263158
--------------------- 2437 -----------------------
"Our house is still 
['swedish', 'greta thunberg', 'world economic forum']
DAVOS — U.S. Preside
['us', 'donald trump', 'greta thunberg']
0.3125
--------------------- 2438 -----------------------
Greta Thunberg is kn
['greta thunberg', 'swedine', 'stockholm']
Greta Thunberg, a fr
['greta thunberg', 'greta inc', 'keean bexte']
0.23529411764705882
--------------------

--------------------- 2480 -----------------------
Germany has begun to
['germany', 'austria', 'france']
Mar 16, 2020 , 8:37A
['8:37am germany', 'air pics germany', 'france']
0.4166666666666667
--------------------- 2481 -----------------------
"Help must also be w
['bundeswehr', 'iraq', 'ktcc']
With the situation s
['iraq', 'iran', 'bundeswehr']
0.16
--------------------- 2482 -----------------------
With Judowüfen and K
['judowüfen', 'kartactiffen', 'bond girl pussy galore']
Honor Blackman, best
['honor blackman', 'james bond', 'pussy galore']
0.14
--------------------- 2483 -----------------------
ESA-Mission  + © Pic
['esa mission  + © picture alliance', 'mercury', 'bepicolombo']
Earth To Be Buzzed B
['esa', 'atg', 'earth']
0.25
--------------------- 2484 -----------------------
Knowledge gap at the
['the robert koch institut bernd murawski', 'the united kingdom', 'imperial college london']
South Korea’s Center
['south korea’s', 'centers for disease control and prevention', 'daegu'

--------------------- 2524 -----------------------
After the emperor of
['palestine', 'mashreq', 'the white house']
The Kremlin said on 
['kremlin', 'russia', 'us']
0.30158730158730157
--------------------- 2525 -----------------------
"The exhibition, whi
['red expeditions', 'britain', 'corona']
British Prime Minist
['british', 'boris johnson', 'twitter']
0.1111111111111111
--------------------- 2526 -----------------------
The Public Security 
['the public security sector', 'qena', 'qena sahrawi sahrawi']
Subscribe to receive
['qena', 'al wakar', 'abucott']
0.3
--------------------- 2527 -----------------------
Despite her free fro
['corona', 'yemen', 'yemen']
Follow-up March 20 T
['the joint technical committee', 'corona', 'abdul hakim al kahlani']
0.35714285714285715
--------------------- 2528 -----------------------
He said in a dialogu
['german', 'der spiegel', 'iranian']
Iranian Foreign Mini
['iranian', 'mohammad jawad zarif', 'europe']
0.5714285714285714
--------------------- 2

--------------------- 2574 -----------------------
In Sakarya, two brot
['sakarya', 'the national solidarity campaign', 'the national solidarity campaign, misra']
Donated half of the 
['the national solidarity campaign', 'hüseyin aydin', 'keban']
0.16666666666666666
--------------------- 2575 -----------------------
According to the Dep
['the department of communication', 'oktay interior', 'süleyman soylu']
AA President of Pres
['fuat oktay', 'interior', 'süneyman soylu']
0.21428571428571427
--------------------- 2576 -----------------------
22.02.2020 13:16 | L
['antalya', 'halil özçelik', 'behçet']
A different diagnosi
['sspe', 'sspe', 'özçelik']
0.125
--------------------- 2577 -----------------------
We need to solve the
['idlib', 'russia', 'moscow']
President Recep Tayy
['recep tayyip erdogan', 'azerbaijan', 'libya']
0.1
--------------------- 2578 -----------------------
Gümüşhane quarantine
['gümüşhane', 'gumushane', 'belgium']
Fenerbahce's Goalkee
['fenerbahce goalkeeper berke o

In [33]:
text_key_df.isna().sum()

pair_id             0
url1_lang           0
url2_lang           0
text1               0
text2               0
translated_body1    0
translated_body2    0
key1                0
key2                0
key_score           0
dtype: int64

In [34]:
text_key_df[text_key_df['translated_body1'].isna()]

,pair_id,url1_lang,url2_lang,text1,text2,translated_body1,translated_body2,key1,key2,key_score


In [35]:
text_key_df

,pair_id,url1_lang,url2_lang,text1,text2,translated_body1,translated_body2,key1,key2,key_score
0,1484084337_1484110209,en,en,"MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove...","MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove...","[martinsburg, wva, new year’s day, west virgin...","[haiti, haitian, jovenel moïse, moïse, the nat...",0.045455
1,1576314516_1576455088,en,en,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,"[india, zomato, china, ant financial, zomato, ...","[india, google, boston consulting group, bcg, ...",0.121951
2,1484036253_1483894099,en,en,BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...,BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...,"[bengaluru, reuters, india, indian space resea...","[india, indian space research organisation, k ...",0.318182
3,1484034982_1483785560,en,en,Asserting that India too should protect the in...,The air quality recorded at 9.38 am was 433 in...,Asserting that India too should protect the in...,The air quality recorded at 9.38 am was 433 in...,"[india, malaysia, indonesia, sea, indonesia, c...","[met, meteorological]",0.000000
4,1484188439_1484378177,en,en,From The Guardian\n\nFrom Boeing to Whole Food...,Police have arrested a 38-year-old black man f...,From The Guardian\n\nFrom Boeing to Whole Food...,Police have arrested a 38-year-old black man f...,"[guardian, boeing, whole foods, donald trump, ...","[orange country, ny, jews, hasidic, monsey, ha...",0.031250
...,...,...,...,...,...,...,...,...,...,...
2585,1586195445_1598778991,tr,tr,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...",BM'den Yemen'de kadınların doğumda ölüm riski ...,"The UN announced that more than 2,000 domestic...","In Yemen, Women's Death Risk Warning Explanati...","[un, aden, the united nations, un, yemen, the ...","[yemen, women death risk warning explanation, ...",0.300000
2586,1590915424_1590940388,tr,tr,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"Because of the Kovid-19, the Survival Struggle...",The new type of coronavirus (Kovid-19) is cons...,"[la liga clubs, football league, la liga, span...","[football league, la liga, spanish, the cadena...",0.857143
2587,1526157103_1492737005,tr,tr,\n\n\n\n\n\n\n\nİflas noktasındaki kulüplerin ...,"TFF, resmi internet sitesinden Beşiktaş'ın fai...",It is stated that the sales of the clubs at th...,TFF has published an explanation on the offici...,"[besiktas, ahmet nur dek, turkey banks associa...","[tff, besiktas, team spend limits, ahmet nur r...",0.090909
2588,1603274500_1618292937,tr,tr,Ergene Belediyesi yol çalışmalarına aksatmadan...,Ergene'de Ahimehmet ve Yeşiltepe mahallelerind...,Ergene Municipality continues without disrupti...,"In Ergene, the mask was distributed in Ahimehm...","[ergene municipality, tekirdag, ergene municip...","[ergene, ergene county, ergene, ergene, ergene...",0.166667


In [ ]:
path = 'eval/_EVAL_text_named-entity_score.csv'
title_key_df.to_csv(path,index=False)